In [1]:
from functools import partial
from itertools import product
import glob
import numpy as np
import os
import pandas as pd
import pybedtools as pbt
import statsmodels
import subprocess
import sys

sys.path.append("/dors/capra_lab/users/fongsl/tools/py_/")
import config_readwrite as crw
sys.path.append("/dors/capra_lab/users/fongsl/tools/genome/")

import fet
import chr_functions
import split_filename

In [2]:
name = "/data/hodges_lab/ATAC-STARR_B-cells/bin_human-evolution/config"
config, configfile_name = crw.read_config(name)

SHUF_PATH = config["SHUFFLES"]["path"]
REGIONS = config["CIS_TRANS_LIFTOVER"]["regions"]
REGION_ANNOT = config["CIS_TRANS"]["regions_annotations"]

CL = "LCL8664"

ID_TAG = config["TF_FOOTPRINTING_JASPAR"]["ID_TAG"]
print(ID_TAG)
FP_RE = config[f"TF_FOOTPRINTING_JASPAR_{CL}"]["FP"]
FP_OR_RE = config[f"TF_FOOTPRINTING_JASPAR_{CL}"]["FP_OR"] # write
FP_SHUF_OR_RE = config[f"TF_FOOTPRINTING_JASPAR_{CL}"]["FP_SHUF_OR"]  # write
FP_HH_OR_RE = config[f"TF_FOOTPRINTING_JASPAR_{CL}"]['FP_HH_OR']  # write
FP_MM_OR_RE = config[f"TF_FOOTPRINTING_JASPAR_{CL}"]['FP_MM_OR']  # write


RE = config["TF_FOOTPRINTING"]["results"]


path, region_file, region = split_filename.split_filename(REGIONS)

MA


In [3]:
fp = pd.read_csv(FP_RE, sep = '\t')

regions = pd.read_csv(REGION_ANNOT, sep ='\t',low_memory=False)

# Functions 

1. get sets of region ids for testing enrichment. 
2. get counts of FP in test regions v. FP not in test regions

In [4]:
def get_fp_counts(test, df):
    
    """
    return sets of region ids that overlap/do not overlap test column.
    """
    
    test_region_ids = set(df.loc[df[test]==1, "region_id"])
    nottest_region_ids = set(df.loc[df[test]!=1, "region_id"])
    
    return test_region_ids, nottest_region_ids

In [5]:
def make_2x2_among_active(test_tf, tf, ytest, ntest):
    
    """
    return counts of TF FP 
    
    input
        test_tf (dataframe)
        tf (str)
        ytest (list) - region ids w/ test label
        ntest (list) - region ids w/o test label
    """
    
    if len(set(test_tf[tf]))>1:  # check that TF footprints at all. 

        # differential FP in yes test set?
        first_set = set(test_tf.loc[test_tf["region_id"].isin(ytest), tf])
        
        if len(first_set)>1:

            b,a = test_tf.loc[test_tf["region_id"].isin(ytest)].groupby(tf).count().reset_index().iloc[:, 1]
            
        elif False in first_set:
            b,a = len(ytest), 0  # all values are False in y set
        
        else:
            b,a = 0, len(ytest)  # all values are True in y set

        # differential FP in no test set?  
        second_set = set(test_tf.loc[test_tf["region_id"].isin(ntest), tf])

        if len(second_set)>1:   
            d,c = test_tf.loc[test_tf["region_id"].isin(ntest)].groupby(tf).count().reset_index().iloc[:, 1]

        elif False in second_set:
            d,c = len(ntest), 0  # all values are False in n set
            
        else:
            d,c = 0, len(ntest) # all values are True in n set
            
    else:
        a,b,c,d = 0,0,0,0
    
    return a,b,c,d
    

# make a list of TFs

In [6]:
#col = 'HH-active_MM-inactive_cis+trans'

tfs = list(fp)[4:]  #list of TF names to test

len(tfs)

746

# compare with shuffles
- this takes 6h to run!
- compare whether footprints are enriched in active categories v. flanking. 
- This is only 10 shuffles in flanking, sharedAccessible regions. 
- Not sure I would expect we would see a difference in TF footprinting between shuffles and active regions

In [7]:
test_cols = [ 
            "cis", "trans", 
            'HH-active_MM-inactive_MH-inactive_cis',
            'HH-active_MM-inactive_HM-inactive_trans',
            'HH-active_MM-inactive_cis-only',
            'HH-active_MM-inactive_trans-only',
            'HH-active_MM-inactive_cis+trans',
            'MM-active_HH-inactive_HM-inactive_cis',
            'MM-active_HH-inactive_MH-inactive_trans',
            'MM-active_HH-inactive_cis-only',
            'MM-active_HH-inactive_trans-only',
            'MM-active_HH-inactive_cis+trans',
            "cis_only",
            "trans_only", 
            "cis+trans"
                  ]

# Slurm jobs for shuffle enrichment

In [8]:
if os.path.exists(FP_SHUF_OR_RE) is False:
    
    # run slurm job
    for col in test_cols:
        out = os.path.join(RE, f"{CL}-{col}_x_shuf_OR.tsv")

        cmd = f"sbatch /data/hodges_lab/ATAC-STARR_B-cells/bin_human-evolution/TF_FP/run_fp.slurm {col} {CL}"

        if os.path.exists(out) is False:
            subprocess.call(cmd, shell = True)

        else:
            numlines = sum(1 for line in open(out))
            if numlines< 746:

                subprocess.call(cmd, shell = True)
            else:
                print("already run", out)


# merge all the results together.  

In [ ]:
# save the file
fs = glob.glob(os.path.join(RE, f"{CL}-*_x_shuf_OR.tsv"))

# merge all the rheMac 10 regions v. shuffled TF FP OR results
and write file: /data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/rhe_footprinting_shuf_JASPAR_OR.tsv

In [ ]:
merged_results = {}
for i, f in enumerate(fs):

    d = pd.read_csv(f, sep = '\t')
    merged_results[i]=d

results_fpVshuf_OR = pd.concat(merged_results.values()).drop_duplicates()
results_fpVshuf_OR.groupby("tested")["arch_id"].count()

## clean up

In [ ]:
"""
This is why we clean up the dataframe below: 

an example duplicate record
"""
results_fpVshuf_OR.loc[(results_fpVshuf_OR["tested"] == "MM-active_HH-inactive_trans-only")
                                    & (results_fpVshuf_OR["arch_id"] == "THRBVAR.2_MA1575.1")]

In [ ]:
"""
1. drop na's
2. reset the index
3. find index for all duplicate tests per activity category/TF FP (results are the same) 
4. drop duplicate indexes.
5. drop index column
6. add TF column
6. save unique results

"""

#1
results_fpVshuf_OR = results_fpVshuf_OR.loc[~results_fpVshuf_OR["reject_null"].isna()] 

#2
results_fpVshuf_OR = results_fpVshuf_OR.reset_index() # reset the index

#3
drop = []
for act in set(list(results_fpVshuf_OR["tested"])):
    for tf in set(list(results_fpVshuf_OR["arch_id"])):
        test = results_fpVshuf_OR.loc[(results_fpVshuf_OR["tested"] == act)
                                    & (results_fpVshuf_OR["arch_id"] == tf)]

        if len(test)>1:  # find duplicates
            drop.append(test.index[-1]) # get all the indexes to drop

#4
d = results_fpVshuf_OR.drop(drop)

print(d.groupby("tested")["arch_id"].count())

#5
d = d.drop(columns = ["index"])

#6
d["TF"] = d["arch_id"].apply(lambda x: x.split("_")[0])

#7
d.to_csv(FP_SHUF_OR_RE, sep = '\t',index=False)

print(results_fpVshuf_OR.shape, d.shape, len(set(drop)))

d.head()

In [ ]:
set(re["TF"])

In [ ]:
CL = "GM12878"
FP_SHUF_OR_RE = config[f"TF_FOOTPRINTING_JASPAR_{CL}"]["FP_SHUF_OR"]  # write

g = pd.read_csv(FP_SHUF_OR_RE,sep='\t')

set(g["TF"])

In [ ]:
re.head()

In [ ]:
re.loc[re["TF"].str.contains("ALX1")]